In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, Concatenate
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

batch_size = 32
latent_dim = 128
num_samples = 40000
epochs = 50
validation_split = 0.2

# Load data
input_texts, target_texts = [], []
input_chars, target_chars = set(), set()

with open('./fra.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

for line in lines[:min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    input_chars.update(input_text)
    target_chars.update(target_text)

input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

print(f'Number of samples: {len(input_texts)}')
print(f'Number of unique input tokens: {num_encoder_tokens}')
print(f'Number of unique output tokens: {num_decoder_tokens}')
print(f'Max sequence length for inputs: {max_encoder_seq_length}')
print(f'Max sequence length for outputs: {max_decoder_seq_length}')

# Vectorize data
input_token_index = dict((char, i) for i, char in enumerate(input_chars))
target_token_index = dict((char, i) for i, char in enumerate(target_chars))

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length), dtype='int32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length), dtype='int32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t] = input_token_index[char]
    for t, char in enumerate(target_text):
        decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Define the model
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
encoder_bi_lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(
    LSTM(latent_dim, return_state=True, return_sequences=False))(encoder_embedding)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Train the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['acc'])
early_stop = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint('model_weights.keras', monitor='val_loss', save_best_only=True)
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=validation_split,
        callbacks=[early_stop, checkpoint])

# Define encoder and decoder models for inference
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim * 2,))
decoder_state_input_c = Input(shape=(latent_dim * 2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Reverse token index dictionaries
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

# Decode sequence
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['\t']

    decoded_sentence = ''
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

# Test the model
input_sentences = []
decoded_sentences = []
separator_lines = []
french_to_english_output = []
english_to_french_output = []

for seq_index in range(10, 1000, 10):
    input_seq = encoder_input_data[seq_index:seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    input_sentences.append(input_texts[seq_index])
    decoded_sentences.append(decoded_sentence)
    separator_lines.append('-')

    # Store outputs for both directions
    french_to_english_output.append((input_texts[seq_index], decoded_sentence))
    english_to_french_output.append((decoded_sentence, input_texts[seq_index]))





In [9]:
print("\nFrench to English Translations:")
for input_sent, decoded_sent in french_to_english_output:
    print(f'French: {decoded_sent} -> English: {input_sent}')

# Print English to French outputs
print("\nEnglish to French Translations:")
for decoded_sent, input_sent in english_to_french_output:
    print(f'English: {input_sent} -> French: {decoded_sent}')


French to English Translations:
French: Attendez !
 -> English: Wait!
French: Arrant.
 -> English: Cheers!
French: Atrefais.
 -> English: Hop in.
French: Je suis de moi.
 -> English: I'm OK.
French: Pars de moi.
 -> English: No way!
French: Demande de moi.
 -> English: Ask Tom.
French: Sois autresse.
 -> English: Be fair.
French: Appelle-moi.
 -> English: Call me.
French: Venez.
 -> English: Come on.
French: Sortes.
 -> English: Get out.
French: Va fais.
 -> English: Go away.
French: Il mais.
 -> English: He runs.
French: J'ai en chais.
 -> English: I tried.
French: Je suis tout.
 -> English: I'm sad.
French: Sais-toi !
 -> English: Kiss me.
French: Fermez-le.
 -> English: Shut up!
French: Réveillez-vous !
 -> English: Wake up!
French: Nous avons dépourdis.
 -> English: We lost.
French: Suis-je maintent ?
 -> English: Am I fat?
French: Sois conticieux.
 -> English: Be still.
French: Dis-nous !
 -> English: Drive on.
French: Sortiez !
 -> English: Get real!
French: Passez le problesse.